In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [ ]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [ ]:
!unzip -q train.zip -d .

**Make folders for dataset**

In [ ]:
import os
import tensorflow as tf
import shutil

print( len( os.listdir('/content/train/') ) ) #count the number of img files

#move files into either cat or dog folder

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'): #i = file name like cat1.jpg
  if 'cat' in i:
    shutil.copyfile( '/content/train/' + i, '/content/dataset/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/' + i, '/content/dataset/dog/' + i )


# tf.keras.preprocessing.image.dataset_from_directory()



**Preprocess the tensor data**

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64, #64 images at once, update w 
    subset='training', #training has 80% of data
    validation_split=0.2, #20% of data for validation dataset, use 80% of data for training
    seed=1234,
)

#train_ds format = ( (xxxx = 64x64 image array), (yyyyy = 0 or 1 for dog/cat) )

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64, 
    subset='validation', #validation has 20% of data
    validation_split=0.2, 
    seed=1234,
)

print(train_ds)

def preprocess_func(image_data, key): #introduce preprocessor to increase flops and find optimized w quicker
  image_data = tf.cast(image_data / 255.0, tf.float32 )
  return image_data, key

train_ds = train_ds.map(preprocess_func) #compress input data into range 0-1 rather than using 0-255
val_ds = val_ds.map(preprocess_func)

# import matplotlib.pyplot as plt

# for i, answer in train_ds.take(1): 
#   print(i) 
#   print(answer)
#   plt.imshow( i[0].numpy().astype('uint8') )
#   plt.show()

**Make a model**

In [ ]:
model = tf.keras.Sequential([
    
    #Augment image before putting it into model
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64,64,3) ),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),

    tf.keras.layers.Conv2D( filters=32, kernel_size=(3,3), padding="same", activation='relu' ), #image size = 64x64, color = 3 for rgb
    tf.keras.layers.MaxPooling2D( pool_size=(2,2) ),
    
    tf.keras.layers.Conv2D( filters=64, kernel_size=(3,3), padding="same", activation='relu' ), #image size = 64x64, color = 3 for rgb
    tf.keras.layers.MaxPooling2D( pool_size=(2,2) ),
    
    tf.keras.layers.Dropout(0.2), #to prevent overfitting, remove 20% of previous layer's nodes
    
    tf.keras.layers.Conv2D( filters=128, kernel_size=(3,3), padding="same", activation='relu' ), #image size = 64x64, color = 3 for rgb
    tf.keras.layers.MaxPooling2D( pool_size=(2,2) ),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(128, activation="relu"),
    
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1, activation="sigmoid") #sigmoid used to have one node for last layer = probability of dog or cat
])

model.summary()
model.compile( loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'] )
model.fit(train_ds, validation_data=val_ds, epochs=5)


**Image Augmentation**

In [ ]:
#Augment image using image data generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20, #rotation
    zoom_range=0.15, #zoom
    width_shift_range=0.2, #shift
    height_shift_range=0.2,
    shear_range=0.15,  #shear
    horizontal_flip=True, #flip
    fill_mode="nearest", #how to fill in padding
)

train_dataset = train_generator.flow_from_directory( #behaves like image_dataset_from_directroy()
    '/content/dataset',
    class_mode='binary', #two = binary, more = categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=64,
    target_size=(64,64),
)

val_generator = ImageDataGenerator(rescale=1./255) #no need to augment the validation dataset

val_dataset = val_generator.flow_from_directory(
    '/content/dataset',
    class_mode='binary', #two = binary, more = categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=64,
)

**Save a model or checkpoint after training**

In [ ]:
import tensorflow as tf 
import numpy as np

(trainX, trainY), (testX, testY) = tf.keras.datasets.fashion_mnist.load_data()

trainX = trainX / 255.0
testX = testX / 255.0

trainX = trainX.reshape( (trainX.shape[0], 28,28,1) )
testX = testX.reshape( (testX.shape[0], 28,28,1) )

model = tf.keras.Sequential([
    tf.keras.layers.Flatten( input_shape=(28,28,1) ),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

call_back = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoint/mnist', #mnist{epoch} to store every epoch instead of overwriting and storing only last epoch
    monitor='val_acc',
    mode='max',

    # save checkpoint when val_acc is max

    save_weights_only=True,
    save_freq='epoch'
)

model.summary() #always require input_shape on first layer of the model

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=3, callbacks=[call_back]) #run callbacks in between each epoch to store the weights of the model = save checkpoint

model.evaluate(testX, testY)

In [ ]:
model.save('newFolder/model1') #to save the entire model

In [ ]:
loadedModel = tf.keras.models.load_model('newFolder/model1')
loadedModel.summary()

loadedModel.evaluate(testX, testY)

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten( input_shape=(28,28,1) ),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model2.summary()

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

model2.load_weights('checkpoint/mnist')

model2.evaluate(testX, testY)


**Tesorboard visulization, test automization usig EarlyStoppig**

In [ ]:
import tensorflow as tf 
import numpy as np

(trainX, trainY), (testX, testY) = tf.keras.datasets.fashion_mnist.load_data()

trainX = trainX / 255.0
testX = testX / 255.0

trainX = trainX.reshape( (trainX.shape[0], 28,28,1) )
testX = testX.reshape( (testX.shape[0], 28,28,1) )

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

from tensorflow.keras.callbacks import TensorBoard
import time

tensorboard = TensorBoard( log_dir='logs/{}'.format( 'first model' + str( int(time.time()) ) ))

#EarlyStoppig
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping( monitor='val_accuracy', patience=3, mode='max' ) #after 3 epochs, stop training if there is no improvemt in val_acc, mode = 'min' for val_loss

model.fit(trainX, trainY, validation_data=(testX, testY), epochs=300, callbacks=[tensorboard, es]) #run callbacks after each training



In [ ]:
%load_ext TensorBoard

In [ ]:
%tensorboard --logdir logs

**Make a model with Functional API instead of Sequential**

In [ ]:
import tensorflow as tf 
import numpy as np

(trainX, trainY), (testX, testY) = tf.keras.datasets.fashion_mnist.load_data()

trainX = trainX / 255.0
testX = testX / 255.0

trainX = trainX.reshape( (trainX.shape[0], 28,28,1) )
testX = testX.reshape( (testX.shape[0], 28,28,1) )

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True) #preview a model

In [ ]:
input1 = tf.keras.layers.Input(shape=[28, 28])
flatten1 = tf.keras.layers.Flatten()(input1) #input1 -> flatte1
dense1 = tf.keras.layers.Dense(28*28, activaton='relu')(flatten1)
reshape1= tf.keras.layers.Reshape( (28,28) )(dense1) #reshape size should be equal to previous layer's number of nodes

concat1 = tf.keras.layers.Concatenate()([input1, reshape1]) 
flatten2 = tf.keras.layers.Flatten()(cocat1)
output = tf.keras.layers.Dense(10, activation='softmax')(flatten2)

model = tf.keras.Model(input1, output) #make a model with input and output

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True) #preview a model

**Transfer learning**

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .
!unzip -q train.zip -d . 

In [ ]:
import os 
import tensorflow as tf 
import shutil

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/dog/' + i )

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

def preprocess_func(image_data, key): #introduce preprocessor to increase flops and find optimized w quicker
  image_data = tf.cast(image_data / 255.0, tf.float32 )
  return image_data, key

train_ds = train_ds.map(preprocess_func) #compress input data into range 0-1 rather than using 0-255
val_ds = val_ds.map(preprocess_func)

1. Download Inception_v3.h5, weight file

In [ ]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

2. import Google's InceptionV3 model

In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3( input_shape=(150,150,3), include_top=False, weights=None ) 
#originally InceptioV3 takes input_shape=(299,299,3)
#top layer = last output layer = last dense layer

3. integrate weight file and InceptionV3

In [ ]:
inception_model.load_weights('inception_v3.h5')

inception_model.summary()

4. Set layers not trainable 



In [7]:
#No fine tuning
for i in inception_model.layers:
  i.trainable = False

#fine tuning = train inception model from mixed6 layer
unfreeze = False
for i in inception_model.layers:
  if i.name == 'mixed6':
    unfreeze = True
  if unfreeze == True:
    i.trainable = True

last_layer = inception_model.get_layer('mixed7')

In [14]:
import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(last_layer.output)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation='sigmoid')(drop1) #1 for dog and cat classification

model = tf.keras.Model(inception_model.input, layer3)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2
313/313 [==============================] - 1287s 4s/step - loss: 0.1610 - acc: 0.9524 - val_loss: 0.0872 - val_acc: 0.9622
Epoch 2/2
313/313 [==============================] - 1220s 4s/step - loss: 0.0404 - acc: 0.9843 - val_loss: 0.0890 - val_acc: 0.9700


fine tuning = update weight by little = small learning rate in optimizer

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['acc'])
model.fit(train_ds, validation_data=val_ds, epochs=2)